# Lab 4

## Init data

### Task and init data tast

![Task](image.png)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate as tb

X = np.array([i+1 for i in range(5)])
Y = np.array([2 + 0.5 * i for i in range(5)])

lenData = len(X)
nan = np.nan
fmt_number = '%.3f'

Data = np.array(
    [
        [nan, nan, nan, 3, 4],
        [nan, nan, 5, 3, 5],
        [nan, 3, 6, 4, nan],
        [4, 4, 2, 2, nan],
        [1, 4, nan, nan, nan]
    ]
)

ny_SumsOfYInData = np.array([np.nansum(Data[i, :]) for i in range(lenData)])
nx_SumsOfYInData = np.array([np.nansum(Data[:, i]) for i in range(lenData)])


### Init functions

#### to_strings

In [ ]:
def to_strings(a: np.ndarray):
    tp = type('')
    res = a.copy()
    match(res.ndim):
        case 1: res = np.array([str(np.round(x, 3)) for x in a], tp)
        case 2:
            for i in range(a.shape[0]):
                temp = np.array([str(np.round(x, 3)) for x in res[i]], tp)
                res[i] = temp
        case _:
            raise Exception('Unhandled ndarray dimentails amount')

    return res


#### tabulate

In [ ]:
def tabulate(a: np.ndarray, col_headers: np.ndarray = [], row_headers: np.ndarray = []):
    a_copy = a.copy()
    row_amount = a_copy.shape[0]
    if (len(row_headers) != 0 and row_amount != len(row_headers)):
        raise Exception(
            "Length of rows headers array must be equal input array rows count")
    match(len(a.shape)):
        case 1: raise Exception("Array must have more than one dimention")
        case 2:
            col_amount = a_copy.shape[1]
            colheaders_count = len(col_headers)
            if (colheaders_count > 0):
                col_amount += 1
            if (colheaders_count != 0 and col_amount != colheaders_count):
                raise Exception(
                    "Length of columns headers array must be equal input array columns count")
            if (len(row_headers) != 0):
                res = [[]
                       for i in range(row_amount)]
                for i in range(row_amount):
                    res[i].append(row_headers[i])
                    for x in to_strings(a_copy[i]):
                        res[i].append(x)
            else:
                res = a_copy
        case _:
            raise Exception('Unhandled ndarray shape situation')

    print(tb(res, headers=col_headers,
          tablefmt='rounded_grid', numalign='center'))


### Data output

In [ ]:
tabulate(Data, np.concatenate([[
    '''   \  X
Y  \\'''], to_strings(X)]), to_strings(Y))


## Correlation field

In [ ]:
# trend line
start = (np.flip(X)[0] + np.flip(Y)[0]) / 2
qr = (((np.flip(X)[-1] + np.flip(Y)[-1]) / 2) - start) / lenData

tl = [start + qr * i for i in range(lenData)]
values_y_notnan = [None] * lenData

plt.plot(np.flip(Y), X, label='Y / X')
plt.plot(np.flip(X), Y, label='X / Y')
plt.plot(tl, Y, label='Trend line', c='gray', ls='--')
for i in range(lenData):
    values_y_notnan[i] = Y[~np.isnan(Data[:, i])]
    l = len(values_y_notnan[i])
    plt.scatter(np.repeat(X[i], l), values_y_notnan[i], c='black')


plt.title('Correlation field')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()


## Least squares regression line equation

### Plot

In [ ]:
y_avarage_xj = np.zeros(lenData)
values_data_notnan_byX = [None] * lenData
values_data_notnan_byY = [None] * lenData

for i in range(lenData):
    q = Data[:, i]
    q1 = Data[i, :]
    values_data_notnan_byX[i] = q[~np.isnan(q)]
    values_data_notnan_byY[i] = q1[~np.isnan(q1)]
    y_avarage_xj[i] = np.sum(values_data_notnan_byX[i]) * \
        Y[i] / nx_SumsOfYInData[i]

plt.scatter(X, y_avarage_xj, c="black")
plt.plot(X, Y, label='X on Y')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
# NOTE: if otklonenia znachitelni -> change the meaning of text below
print('''
Видно, что отклонения точек от построенной
прямой незначительны. Следовательно, связь между признаками
X и Y может носить линейный характер
''')


### The least squares method

#### Table

In [ ]:
nxy = np.sum(nx_SumsOfYInData) + np.sum(ny_SumsOfYInData)

res = np.zeros((lenData + 6, lenData))

print(values_y_notnan)

for i in range(lenData):
    res[i] = Data[i]
res[lenData] = nx_SumsOfYInData
res[lenData + 1] = ny_SumsOfYInData
res[lenData + 2] = [nx_SumsOfYInData[i] * X[i] for i in range(lenData)]
res[lenData + 3] = [ny_SumsOfYInData[i] * Y[i] for i in range(lenData)]
res[lenData + 4] = [nx_SumsOfYInData[i] * X[i]**2 for i in range(lenData)]
res[lenData + 5] = [np.sum([X[i] * values_y_notnan[i][j] * values_data_notnan_byX[i][j]
                            for j in range(len(values_y_notnan[i]))]) for i in range(lenData)]


In [ ]:
cols = np.append([
    ''' \ X
Y \\'''], to_strings(X))
rows = np.append(to_strings(
    Y), ['nx', 'ny', 'nx * x', 'ny * y', 'nx * x^2', 'nxy_xy'])

tabulate(res, cols, rows)

#### The equation

In [ ]:
qr = np.sum(res[lenData + 2])
m = np.array([[nxy, qr],
             [qr, np.sum(res[lenData + 4])]])
v = np.array([np.sum(res[lenData + 3]),
              np.sum(res[lenData + 5])])

solved_equation_koefs = np.linalg.solve(m, v)


def linear_regression_equation(x: float):
    return solved_equation_koefs[0] * x + solved_equation_koefs[1]


In [ ]:
data_maximum = np.max([np.max(values_data_notnan_byX[i])
                      for i in range(lenData)])
indices_info = np.where(Data == data_maximum)
data_maximum_x, data_maximum_y = indices_info[0][0], indices_info[1][0]
x_step = np.average(np.diff(X))
y_step = np.average(np.diff(Y))
c1 = data_maximum_x
c2 = data_maximum_y
U = np.arange(data_maximum_x + 1 - lenData, lenData - data_maximum_x)
V = np.arange(data_maximum_y + 1 - lenData, lenData - data_maximum_y)

u_avarage = np.sum([np.sum(values_data_notnan_byX[i]) *
                    U[i] for i in range(lenData)]) / nxy
v_avarage = np.sum([np.sum(values_data_notnan_byY[i]) *
                    V[i] for i in range(lenData)]) / nxy

u_2_avarage = np.sum([np.sum(values_data_notnan_byX[i]) *
                      U[i]**2 for i in range(lenData)]) / nxy
v_2_avarage = np.sum([np.sum(values_data_notnan_byY[i]) *
                      V[i]**2 for i in range(lenData)]) / nxy

Su = np.sqrt(u_2_avarage - u_avarage**2)
Sv = np.sqrt(v_2_avarage - v_avarage**2)

values_U_notnan = [None] * lenData
values_V_notnan = [None] * lenData

# print(values_data_notnan_byU)

for i in range(lenData):
    values_U_notnan[i] = U[~np.isnan(Data[:, i])]
    values_V_notnan[i] = V[~np.isnan(Data[:, i])]

nujvi_ujvi = np.sum([np.sum([(values_U_notnan[i][j]
                              * values_V_notnan[i][j]
                              * values_data_notnan_byX[i][j]) for j in range(
    len(values_data_notnan_byX[i]))]) for i in range(lenData)])


In [ ]:
rb = (nujvi_ujvi - nxy * u_avarage * v_avarage) / (nxy * Su * Sv)
x_avarage_new = u_avarage * x_step + c1
y_avarage_new = v_avarage * y_step + c2
Sx = Su * x_step
Sy = Sv * y_step


def linear_regression_equation_YonX(x: float):
    return y_avarage_new + rb * (Sy / Sx) * (x - x_avarage_new)


## Checking the tightness of the connection between Y and X

### Task 4

In [ ]:
n = lenData**2
k = n - 2
tak = 1.714  # Приложение
tH = (np.abs(rb) * np.sqrt(k)) / (np.sqrt(1 - rb**2))


In [ ]:
check = '>', ''

if (tH < tak):
    check = '<', ' не'

example_x = 10

print(f'''
    Т.к. tH = {np.round(tH, 3)} {check[0]} {tak} (tak) =>
Выборочный коеффициент линейной корреляции rb{check[1]} отличается от нуля.
    Следовательно, можно считать, что отклонение объёма продукций
и её себестоимости{check[1]} связаны линейной корреляционной зависимостью.
    Из уравнения регрессии видно, что при отклонении от
нормальных значений объёма продукции (xj), например на {example_x} тыс. руб,
отклонение от нормального значения обёма продукции составит примерно {np.round(solved_equation_koefs[0] * example_x, 3)} тыс. руб.
    Фактически отклонение может составить {np.round(linear_regression_equation(example_x), 3)} тыс. руб,
что является результатом воздействия неучтенных в модели факторов, 
не зависящих от отклонений при распределении предприятий.
''')


### Task 5

In [ ]:
Q = np.sum([(Y[i] - y_avarage_new)**2 for i in range(lenData)])
QR = np.sum([(linear_regression_equation_YonX(X[i]) -
            y_avarage_new)**2 for i in range(lenData)])
Qe = Q - QR

FH = (QR * k) / (Qe * (lenData - 1))
FT = 5.5  # Приложение


In [ ]:
check = '>', ''
if (FH < FT):
    check = '<', ' не'

print(f'''
    Т.к. FH = {np.round(FH, 3)} {check[0]} {FT} (FT) =>
Модель линейной регрессии y_x = {np.round(solved_equation_koefs[0], 3)}x + {np.round(solved_equation_koefs[1], 3)}{check[1]} согласуется с опытными данными
''')


In [190]:
print('''
Итак, мы получили два уравнения линий регрессии y на x,
описывающих зависимость между признаками X и Y. При
подстановке в каждое из них опытных значений признака X
убеждаемся в том, что уравнение регрессии, полученное по
методу наименьших квадратов без использования коэффициента
линейной корреляции r, дает лучшие значения признака Y.
''')


Итак, мы получили два уравнения линий регрессии y на x,
описывающих зависимость между признаками X и Y. При
подстановке в каждое из них опытных значений признака X
убеждаемся в том, что уравнение регрессии, полученное по
методу наименьших квадратов без использования коэффициента
линейной корреляции r, дает лучшие значения признака Y.

